In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
sns.set_theme(style="whitegrid")
print("success")

success


In [57]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mirosval/personal-cars-classifieds")

print("Path to dataset files:", path)

# We have chosen some real world dataset that were the user have scraped data for purchashing the best car based on the posting.

Using Colab cache for faster access to the 'personal-cars-classifieds' dataset.
Path to dataset files: /kaggle/input/personal-cars-classifieds


In [58]:
import os

# read csv
df = pd.read_csv(os.path.join(path, "all_anonymized_2015_11_2017_03.csv"))
display(df.info())

/tmp/ipython-input-625179341.py:4: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(path, "all_anonymized_2015_11_2017_03.csv"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552912 entries, 0 to 3552911
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   maker                object 
 1   model                object 
 2   mileage              float64
 3   manufacture_year     float64
 4   engine_displacement  float64
 5   engine_power         float64
 6   body_type            object 
 7   color_slug           object 
 8   stk_year             float64
 9   transmission         object 
 10  door_count           float64
 11  seat_count           float64
 12  fuel_type            object 
 13  date_created         object 
 14  date_last_seen       object 
 15  price_eur            float64
dtypes: float64(8), object(8)
memory usage: 433.7+ MB


None

### Let us check a sample of data

In [59]:

df.head()

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur
0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,NaN,NaN,man,5.0,7.0,diesel,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,NaN,NaN,man,5.0,5.0,diesel,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31
2,bmw,NaN,97676.0,2010.0,1995.0,85.0,NaN,NaN,NaN,man,5.0,5.0,diesel,2015-11-14 18:10:06.861792+00,2016-01-27 20:40:15.46361+00,12065.06
3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,NaN,NaN,man,5.0,5.0,gasoline,2015-11-14 18:10:06.872313+00,2016-01-27 20:40:15.46361+00,2960.77
4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,NaN,NaN,man,5.0,5.0,gasoline,2015-11-14 18:10:06.880335+00,2016-01-27 20:40:15.46361+00,2738.71


### used percentage to better gauge on how many null values there are

In [ ]:
df.isna().mean() * 100


In [ ]:
df.describe()

### Created A Threshold for lots of

In [ ]:
cols_to_drop = []
for col in df.columns:
  if df[col].isna().mean() * 100 > 60:
    cols_to_drop.append(col)

print(cols_to_drop)

In [ ]:
df = df.drop(columns = cols_to_drop)
display(df.info())

In [ ]:
# we have decided to change all floats to median and all object or string to the most common occurence

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].median())
# check null perecentage
df.isna().mean() * 100

In [ ]:
df["date_created"] = pd.to_datetime(df["date_created"], format="mixed")
df["date_last_seen"] = pd.to_datetime(df["date_last_seen"], format="mixed")

# feature engineering
# we are trying to get the ad age of the listing (this was in 2017), we beleive when a car is listed longer it may seem the car is overpriced or has issues.
df["ad_age_days"] = (df["date_last_seen"] - df["date_created"]).dt.days

#  We can now the effiecieny of the car by this values
df["power_per_liter"] = df["engine_power"] / (df["engine_displacement"] / 1000)

# A larger car seems to be a no brainer if the price is good
df["large_car"] = (df["seat_count"] >= 5) & (df["door_count"] >= 4)

df["price_bin_id"] = pd.qcut(df["price_eur"], q=10, labels=False, duplicates="drop")

df["slow_seller"] = (df["ad_age_days"] > 90).astype(int)


In [ ]:
df.info()
df.head(5)

In [ ]:
avg_price_by_model = df.groupby("model")["price_eur"].transform("mean")
print(avg_price_by_model)
df["model"]

In [ ]:
corr = df[['large_car', 'slow_seller', 'engine_displacement', 'engine_power', 'price_eur', "ad_age_days", "price_bin_id"]].corr()

# Create the heatmap
sns.heatmap(corr, annot=True, cmap='viridis')
# 1 -> strong correlation
# 0 -> no relationship
# -1 -> strong negative relationship
# it makes sense as larger engines tend to produce more power.
# other corrs dont seem to have any relationship

# Show the plot
plt.show()
print(corr)

In [ ]:
df.groupby("price_bin")["ad_age_days"].mean().plot(
    kind="bar",
    rot=45,
    title="Average Ad Age by Price Bin"
)

